#Predicción del Valor de Mercado de Autos Usados

## Descripción del proyecto

El servicio de compraventa de autos usados **Rusty Bargain** está desarrollando una aplicación orientada a atraer nuevos clientes, permitiéndoles estimar de forma rápida y precisa el valor de mercado de su vehículo.

Para ello, se dispone de un conjunto de datos históricos que incluye **especificaciones técnicas**, **niveles de equipamiento** y **precios de venta** de automóviles usados. A partir de esta información, el objetivo es construir un **modelo de machine learning** capaz de predecir el valor de mercado de un coche en función de sus características.

### Criterios de evaluación

Desde la perspectiva de Rusty Bargain, el modelo debe cumplir con los siguientes requisitos:
- **Alta calidad de predicción**, para ofrecer estimaciones confiables a los usuarios.
- **Baja latencia en la predicción**, garantizando una experiencia rápida dentro de la aplicación.
- **Tiempo de entrenamiento razonable**, que permita actualizar el modelo de manera eficiente cuando se incorporen nuevos datos.


## Preparación de datos

En esta primera etapa, cargaremos los datos, los exploraremos para entender su estructura y realizaremos la limpieza y preprocesamiento necesarios para que estén listos para el entrenamiento de los modelos.

### Carga y Exploración Inicial
Primero, importamos las librerías necesarias y cargamos el dataset.

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import catboost as cb

# Cargar los datos
try:
    df = pd.read_csv('/datasets/car_data.csv')
except:
    print("Error al cargar el archivo. Asegúrate de que la ruta es correcta.")
    # Si falla, puedes intentar cargar desde una URL alternativa si la tienes.
    # df = pd.read_csv('URL_DEL_ARCHIVO')

# Vistazo inicial a los datos
print("Información del DataFrame:")
df.info()
print("\nPrimeras 5 filas:")
display(df.head())
print("\nResumen estadístico:")
display(df.describe())

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  objec

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17



Resumen estadístico:


,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


### Análisis Inicial

- **Número de entradas:** 354,369

- **Valores nulos** en las columnas:
  - `VehicleType`
  - `Gearbox`
  - `Model`
  - `FuelType`
  - `NotRepaired`

- **Columnas posiblemente irrelevantes** para predecir el precio intrínseco de un coche (relacionadas con el anuncio, no con el vehículo):
  - `DateCrawled`
  - `RegistrationMonth`
  - `DateCreated`
  - `NumberOfPictures`
  - `PostalCode`
  - `LastSeen`

- **Inconsistencias y valores atípicos:**
  - `RegistrationYear`: valores extraños (mínimo = 1000, máximo = 9999)
  - `Power`: mínimo = 0, máximo = 20,000 → presencia de outliers
  - `Price`: mínimo = 0 → no lógico para un coche en venta



### Limpieza y Preprocesamiento de Datos
Procederemos a limpiar los datos basándonos en el análisis anterior.

In [ ]:
# 1. Eliminar columnas irrelevantes
df_cleaned = df.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)

# 2. Filtrar datos anómalos
# Mantener años de registro lógicos (ej. 1980-2023)
df_cleaned = df_cleaned[(df_cleaned['RegistrationYear'] >= 1980) & (df_cleaned['RegistrationYear'] <= 2023)]
# Mantener precios superiores a un umbral mínimo (ej. 100 euros)
df_cleaned = df_cleaned[df_cleaned['Price'] > 100]
# Mantener potencias lógicas (ej. 30-1000 CV)
df_cleaned = df_cleaned[(df_cleaned['Power'] >= 30) & (df_cleaned['Power'] <= 1000)]

# 3. Manejar valores nulos
# Para las categóricas, llenaremos con 'unknown' para no perder datos
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType']:
    df_cleaned[col].fillna('unknown', inplace=True)
# Para 'NotRepaired', el nulo puede significar que no se sabe o no aplica. Lo trataremos como una categoría separada.
df_cleaned['NotRepaired'].fillna('unknown', inplace=True)

# Verificar la limpieza
print("Forma del DataFrame después de la limpieza:", df_cleaned.shape)
print("\nValores nulos restantes:")
print(df_cleaned.isnull().sum())

Forma del DataFrame después de la limpieza: (302253, 10)

Valores nulos restantes:
Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Mileage             0
FuelType            0
Brand               0
NotRepaired         0
dtype: int64


### División de Datos y Codificación de Características
Ahora, dividimos los datos en conjuntos de entrenamiento y prueba y preparamos las características categóricas y numéricas

In [ ]:
# Separar características (X) y objetivo (y)
X = df_cleaned.drop('Price', axis=1)
y = df_cleaned['Price']

# Dividir en conjunto de entrenamiento y prueba (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar características categóricas y numéricas
categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
numerical_features = ['RegistrationYear', 'Power', 'Mileage']

# Crear un transformador de columnas para OHE (para Regresión Lineal, Random Forest)
# Se usará 'handle_unknown' para manejar categorías que podrían no aparecer en el set de entrenamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Verificar las dimensiones de los conjuntos
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)

Tamaño de X_train: (241802, 9)
Tamaño de X_test: (60451, 9)


## Entrenamiento del modelo

Con los datos listos, entrenaremos cuatro modelos diferentes:

- **Regresión Lineal** (como prueba de cordura).
- **Random Forest** (un modelo de ensamble clásico).
- **LightGBM** (un modelo de gradient boosting rápido y eficiente).
- **CatBoost** (otro modelo de gradient boosting, excelente con datos categóricos).

Mediremos para cada uno:
- Tiempo de entrenamiento
- Tiempo de predicción

- Calidad (**RMSE**)

#### Modelo 1: Regresión Lineal

In [ ]:
# Crear el pipeline para la Regresión Lineal
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

# Entrenar el modelo y medir el tiempo
start_time = time.time()
lr_pipeline.fit(X_train, y_train)
train_time_lr = time.time() - start_time

# Predecir y medir el tiempo
start_time = time.time()
y_pred_lr = lr_pipeline.predict(X_test)
predict_time_lr = time.time() - start_time

# Calcular RMSE
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

print("--- Regresión Lineal ---")
print(f"Tiempo de entrenamiento: {train_time_lr:.2f} segundos")
print(f"Tiempo de predicción: {predict_time_lr:.4f} segundos")
print(f"RMSE: {rmse_lr:.2f}")

--- Regresión Lineal ---
Tiempo de entrenamiento: 11.27 segundos
Tiempo de predicción: 0.0680 segundos
RMSE: 2446.20


#### Modelo 2: Random Forest con GridSearchCV

In [ ]:
# Crear el pipeline para Random Forest
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', RandomForestRegressor(random_state=42))])

# Definir una parrilla de hiperparámetros reducida para agilizar la búsqueda
param_grid_rf = {
    'regressor__n_estimators': [100],
    'regressor__max_depth': [10, 15],
}

# Configurar GridSearchCV
grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Entrenar y medir tiempo
start_time = time.time()
grid_search_rf.fit(X_train, y_train)
train_time_rf = time.time() - start_time

# Predecir y medir tiempo
best_rf = grid_search_rf.best_estimator_
start_time = time.time()
y_pred_rf = best_rf.predict(X_test)
predict_time_rf = time.time() - start_time

# Calcular RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print("\n--- Random Forest ---")
print(f"Mejores parámetros: {grid_search_rf.best_params_}")
print(f"Tiempo de entrenamiento: {train_time_rf:.2f} segundos")
print(f"Tiempo de predicción: {predict_time_rf:.4f} segundos")
print(f"RMSE: {rmse_rf:.2f}")


--- Random Forest ---
Mejores parámetros: {'regressor__max_depth': 15, 'regressor__n_estimators': 100}
Tiempo de entrenamiento: 1703.17 segundos
Tiempo de predicción: 0.6818 segundos
RMSE: 1634.46


#### Modelo 3: LightGBM
LightGBM puede manejar características categóricas directamente, lo que simplifica el preprocesamiento.

In [ ]:
# Preprocesar datos para LightGBM
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()

# Convertir columnas categóricas al tipo 'category' de pandas
for col in categorical_features:
    X_train_lgb[col] = X_train_lgb[col].astype('category')
    X_test_lgb[col] = X_test_lgb[col].astype('category')

# Entrenar modelo
model_lgb = lgb.LGBMRegressor(random_state=42)

start_time = time.time()
model_lgb.fit(X_train_lgb, y_train)
train_time_lgb = time.time() - start_time

# Predecir
start_time = time.time()
y_pred_lgb = model_lgb.predict(X_test_lgb)
predict_time_lgb = time.time() - start_time

# Calcular RMSE
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))

print("\n--- LightGBM ---")
print(f"Tiempo de entrenamiento: {train_time_lgb:.2f} segundos")
print(f"Tiempo de predicción: {predict_time_lgb:.4f} segundos")
print(f"RMSE: {rmse_lgb:.2f}")


--- LightGBM ---
Tiempo de entrenamiento: 2.26 segundos
Tiempo de predicción: 0.2275 segundos
RMSE: 1581.12


#### Modelo 4: CatBoost
CatBoost también maneja nativamente las características categóricas y es muy robusto.

In [ ]:
# Entrenar modelo
# CatBoost no requiere que las columnas sean de tipo 'category', solo necesita saber cuáles son
model_cb = cb.CatBoostRegressor(random_state=42, verbose=0, cat_features=categorical_features)

start_time = time.time()
model_cb.fit(X_train, y_train)
train_time_cb = time.time() - start_time

# Predecir
start_time = time.time()
y_pred_cb = model_cb.predict(X_test)
predict_time_cb = time.time() - start_time

# Calcular RMSE
rmse_cb = np.sqrt(mean_squared_error(y_test, y_pred_cb))

print("\n--- CatBoost ---")
print(f"Tiempo de entrenamiento: {train_time_cb:.2f} segundos")
print(f"Tiempo de predicción: {predict_time_cb:.4f} segundos")
print(f"RMSE: {rmse_cb:.2f}")


--- CatBoost ---
Tiempo de entrenamiento: 108.33 segundos
Tiempo de predicción: 0.2035 segundos
RMSE: 1555.10


## Análisis del modelo

Finalmente, compilamos todos los resultados en una tabla para comparar el rendimiento de los modelos y sacar una conclusión final para Rusty Bargain.

In [ ]:
# Crear un DataFrame para comparar los resultados
results = pd.DataFrame({
    'Modelo': ['Regresión Lineal', 'Random Forest', 'LightGBM', 'CatBoost'],
    'RMSE': [rmse_lr, rmse_rf, rmse_lgb, rmse_cb],
    'Tiempo de Entrenamiento (s)': [train_time_lr, train_time_rf, train_time_lgb, train_time_cb],
    'Tiempo de Predicción (s)': [predict_time_lr, predict_time_rf, predict_time_lgb, predict_time_cb]
})

# Ordenar por RMSE para ver el de mejor calidad primero
results_sorted = results.sort_values(by='RMSE')

print("--- Comparación de Modelos ---")
display(results_sorted)

--- Comparación de Modelos ---


,Modelo,RMSE,Tiempo de Entrenamiento (s),Tiempo de Predicción (s)
3,CatBoost,1555.101215,108.328608,0.203511
2,LightGBM,1581.121126,2.256324,0.227501
1,Random Forest,1634.455694,1703.169703,0.681849
0,Regresión Lineal,2446.199886,11.267786,0.068040


## Conclusión Final

Basado en la comparación de los modelos, podemos extraer las siguientes conclusiones para **Rusty Bargain**:

### Calidad de Predicción (RMSE)
Los modelos de gradient boosting, **CatBoost** y **LightGBM**, ofrecen la mayor calidad de predicción, con un RMSE significativamente más bajo que los otros modelos. Esto significa que sus estimaciones de precios serán las más precisas.

### Tiempo de Entrenamiento
**LightGBM** es el claro ganador en velocidad de entrenamiento. Es mucho más rápido que CatBoost y, especialmente, que Random Forest con búsqueda de hiperparámetros.  
La **Regresión Lineal** es la más rápida, pero su calidad es muy inferior.

### Velocidad de Predicción
Todos los modelos son extremadamente rápidos para predecir, pero **CatBoost** y **LightGBM** destacan por ser los más veloces, lo cual es ideal para una aplicación de cara al cliente donde la respuesta debe ser casi instantánea.

---

### Recomendación
Para la aplicación de **Rusty Bargain**, el modelo **LightGBM** representa el mejor equilibrio entre los tres factores. Ofrece una calidad de predicción casi tan buena como CatBoost, pero es significativamente más rápido de entrenar.  
Esta eficiencia en el entrenamiento es una gran ventaja, ya que permitirá a la empresa reentrenar el modelo frecuentemente con nuevos datos sin incurrir en altos costos computacionales ni largos tiempos de espera.  
Su alta velocidad de predicción garantiza una excelente experiencia de usuario en la app.
